In [1]:
import numpy as np

# Q1

In [2]:
def find_length(cid):
    cid_str = str(cid)
    return len(cid_str)

def get_digit(cid,digit):
    cid_str = str(cid)
    return int(cid_str[digit-1])

def s_t(cid,t):
    return (get_digit(cid,t)+1)%3

def r_t_plus1(cid,t):
    return (get_digit(cid,t))%2

def find_sequence(cid):
    length = find_length(cid)
    sequence = ''
    for i in range(1,length+1):
        sequence = sequence + 's_' + str(s_t(cid,i)) + ' '
        sequence = sequence + str(r_t_plus1(cid,i)) + ' '
    return sequence

In [3]:
find_sequence(1605864)

's_2 1 s_1 0 s_1 0 s_0 1 s_0 0 s_1 0 s_2 0 '

a.) 1. what can we infer about the structure of the transition matrix?

the transition matrix consists of 3 states, which will result in a 3x3 matrix as the only action available is no action.

from the data given, one possible configuration of a transition matrix is

there is 1 time that we move from s1 to s1
there is 1 time that we move from s1 to s2
there is 1 time that we move from s2 to s0

       from
  s0   s1   s2
[0.0   0.0  1.0 ] s0
[0.0   0.5  0.0 ] s1 to
[0.0   0.5  0.0 ] s2

a.) 2. what can we infer about structure of the reward function

b.) The reward function at s0 is 0, this is because s0 cannot move to any other states

# Q2

## 1.

In [4]:
x=8
y=6
z=4
state_num_reward = ((z+1)%3)+1
state_num_penalty = 0
r_reward = 10
r_penalty = -100
r_transition = -1
p = 0.25+0.5*x/10.0
gamma = 0.2+0.5*y/10.0

print('state_num_reward =',state_num_reward)
print('state_num_penalty =',state_num_penalty)
print('p =',p)
print('gamma =',gamma)

state_num_reward = 3
state_num_penalty = 0
p = 0.65
gamma = 0.5


## 2.

In [5]:
# for simplicity, s11 is renamed to be s0, all other states numbers are the same as the assignment

class Gridmap_mdp:
    def __init__(self,grid_map,state_num_reward,state_num_penalty,r_reward,r_penalty,r_transition,p,gamma):
        #initialize all of the attributes
        self.state_num_reward = state_num_reward
        self.state_num_penalty = state_num_penalty
        self.r_reward = r_reward
        self.r_penalty = r_penalty
        self.r_transition = r_transition
        self.p = p
        self.gamma = gamma
        self.grid_map = grid_map
        
        self.count = np.zeros([4,11])
        self.policy = np.ones([4,11])*0.25
        self.action_value = np.zeros([4,11])
        self.value = np.zeros([11])
        self.locations_dict = self.create_location_dict(self.grid_map)
        self.transition_mat = self.init_transition_mat()
        
    #action: 0=up 1=down 2=left 3=right
    def init_transition_mat(self):
        #create the transition matrix
        transition_mat = []
        for action in ['u','d','l','r']:
            transition_mat.append(self.init_transition_mat_direction(action))
            
        return transition_mat
        
    def create_location_dict(self,grid_map):
        #create a dictionary to map state to coordinates
        locations_dict = {}
        for i in range(grid_map.shape[0]):
            for j in range(grid_map.shape[1]):
                # store as x and y
                locations_dict[grid_map[i][j]] = [j,i]
        return locations_dict
        
    def increment_where(self,from_,direction):
        #find where should the probability be increamented, used to help create transition matrix
        [x,y] = self.locations_dict[from_]

        if direction == 'r':
            #indended direction: right
            if x+1 >= 4 or self.grid_map[y,x+1] == -1:
                return [from_,from_]
            else:
                return [from_,self.grid_map[y,x+1]]

        if direction == 'l':
            #indended direction: left
            if x-1 < 0 or self.grid_map[y,x-1] == -1:
                return [from_,from_]
            else:
                return [from_,self.grid_map[y,x-1]]

        if direction == 'u':
            #indended direction: up
            if y-1 < 0 or self.grid_map[y-1,x] == -1:
                return [from_,from_]
            else:
                return [from_,self.grid_map[y-1,x]]

        if direction == 'd':
            #indended direction: down
            if y+1 >= 4 or self.grid_map[y+1,x] == -1:
                return [from_,from_]
            else:
                return [from_,self.grid_map[y+1,x]]

    def init_transition_mat_direction(self,desired_action):
        #create a transition matrix for only for the desired_action
        action_list = ['u','d','l','r']
        transition_mat = np.zeros([11,11])
        for from_ in range(11):
            for action in action_list:
                update_from, update_to = self.increment_where(from_,action)
                if action == desired_action:
                    transition_mat[update_to,update_from] += self.p
                else:
                    transition_mat[update_to,update_from] += (1-self.p)/3.0
        return transition_mat
    
    def get_reward(self,from_,to_):
        #calculate the reward gained from moving from a state to another
        if to_ == state_num_penalty:
            return self.r_penalty
        elif to_ == state_num_reward:
            return self.r_reward
        else:
            return self.r_transition
        
    def get_possible_state_from(self,state_from,action):
        #find what states are possible from a state and action
        possible_states = []
        #0=up 1=down 2=left 3=right
        transition_mat = self.transition_mat[action]
        for i in range(transition_mat.shape[1]):
            if transition_mat[i,state_from] > 0.0:
                state_to = i
                p_to_that_state = transition_mat[i,state_from]
                possible_states.append(np.array([state_to,p_to_that_state]))

    def sample_mc_episode(self,epsilon=0.0):
        #draw a sample episode from the environment
        self.alpha = 0.5
        initial_state = np.random.choice(9, 1)[0]+2
        state = initial_state
        
        states = []
        actions = []
        rewards = []
        
        return_ = 0
        
        while(state != state_num_penalty and state != self.state_num_reward):
            explore = np.random.choice(2,1,p=[1-epsilon,epsilon])[0]
            action = np.random.choice(4,1,p=self.policy[:,state])[0]
            
            if explore == 1:
                action = np.random.choice(4,1)[0]
            
            next_state = np.random.choice(11,1,p=self.transition_mat[action][:,state])[0]
            reward = self.get_reward(state,next_state)
                            
            states.append(state)
            actions.append(action)
            rewards.append(reward)
            
            return_ += reward
            
            state = next_state
            
        return states,actions,rewards
        
    def calculate_return_from_episode(self,rewards):
        #calculate the return from the generated episode
        return_ = 0
        for i,reward in enumerate(rewards):
            return_ += self.gamma**i * reward
        return return_
            
    def update_mc_episode(self,states,actions,rewards):
        #update the action value of the agent wrt to an episode
        for state,action in zip(states,actions):
            self.count[action,state] += 1
            return_ = self.calculate_return_from_episode(rewards)
            self.action_value[action,state] = ((self.count[action,state]-1)*self.action_value[action,state] + return_)/(self.count[action,state]*1.0)
        
    def mc_policy_eval(self,n,epsilon=0.0):
        #evaluate the action value function of a policy
        for i in range(n):
            if i % 1000 == 0:
                print('performing %sth episode'%(i))
            states,actions,rewards = self.sample_mc_episode(epsilon)
            self.update_mc_episode(states,actions,rewards)
        
    def policy_improve(self):
        #improve the policy from the action value function
        self.policy = np.zeros_like(self.policy)
        for state in range(11):
            v_up = Gridmap.action_value[0,state]
            v_down = Gridmap.action_value[1,state]
            v_left = Gridmap.action_value[2,state]
            v_right = Gridmap.action_value[3,state]

            v_max = np.argmax(np.array([v_up,v_down,v_left,v_right]),0)

            self.policy[v_max,state] = 1
            
    def tell_policy(self):
        #print the policy
        action_word = {0:'up',1:'down',2:'left',3:'right'}
        for state in range(11):
            v_up = Gridmap.action_value[0,state]
            v_down = Gridmap.action_value[1,state]
            v_left = Gridmap.action_value[2,state]
            v_right = Gridmap.action_value[3,state]

            v_max = np.argmax(np.array([v_up,v_down,v_left,v_right]),0)
            
            print('on state =',state,', do action =',action_word[v_max])
            
    def mc_learn(self,n,episode_per_step,epsilon=0.0):
        #learn he gridmap with epsilon greedy montecarlo
        for i in range(n):
            self.action_value = np.zeros([4,11])
            print('===============================')
            print('>',i,'th iter')
            print('===============================')
            self.mc_policy_eval(episode_per_step,epsilon)
            self.policy_improve()

In [6]:
grid_map = np.array([[1,2,3,4],
                    [5,6,-1,7],
                    [-1,8,9,10],
                    [-1,-1,0,-1]])

In [7]:
Gridmap = Gridmap_mdp(grid_map,state_num_reward,state_num_penalty,r_reward,r_penalty,r_transition,p,gamma)

# Monte Carlo learning

In [9]:
states,actions,rewards = Gridmap.sample_mc_episode(0.3)
for state,action,reward in zip(states,actions,rewards):
    print('state =',state,'; action =',action,'; rewarded =',reward)

state = 5 ; action = 2 ; rewarded = -1
state = 5 ; action = 3 ; rewarded = -1
state = 6 ; action = 0 ; rewarded = -1
state = 6 ; action = 1 ; rewarded = -1
state = 6 ; action = 1 ; rewarded = -1
state = 8 ; action = 3 ; rewarded = -1
state = 9 ; action = 1 ; rewarded = -100


In [10]:
Gridmap.action_value

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [11]:
for state in range(11):
    v_up = Gridmap.action_value[0,state]
    v_down = Gridmap.action_value[1,state]
    v_left = Gridmap.action_value[2,state]
    v_right = Gridmap.action_value[3,state]
    
    v_max = np.argmax(np.array([v_up,v_down,v_left,v_right]),0)
    
    print('state =',state,'action =',v_max)

state = 0 action = 0
state = 1 action = 0
state = 2 action = 0
state = 3 action = 0
state = 4 action = 0
state = 5 action = 0
state = 6 action = 0
state = 7 action = 0
state = 8 action = 0
state = 9 action = 0
state = 10 action = 0


In [18]:
Gridmap.mc_learn(5,20000,epsilon=0.3)

> 0 th iter
performing 0th episode
performing 1000th episode
performing 2000th episode
performing 3000th episode
performing 4000th episode
performing 5000th episode
performing 6000th episode
performing 7000th episode
performing 8000th episode
performing 9000th episode
performing 10000th episode
performing 11000th episode
performing 12000th episode
performing 13000th episode
performing 14000th episode
performing 15000th episode
performing 16000th episode
performing 17000th episode
performing 18000th episode
performing 19000th episode
> 1 th iter
performing 0th episode
performing 1000th episode
performing 2000th episode
performing 3000th episode
performing 4000th episode
performing 5000th episode
performing 6000th episode
performing 7000th episode
performing 8000th episode
performing 9000th episode
performing 10000th episode
performing 11000th episode
performing 12000th episode
performing 13000th episode
performing 14000th episode
performing 15000th episode
performing 16000th episode
per

In [19]:
Gridmap.tell_policy()

on state = 0 , do action = up
on state = 1 , do action = left
on state = 2 , do action = right
on state = 3 , do action = up
on state = 4 , do action = left
on state = 5 , do action = right
on state = 6 , do action = right
on state = 7 , do action = right
on state = 8 , do action = left
on state = 9 , do action = right
on state = 10 , do action = right


In [14]:
states, actions, rewards = Gridmap.sample_mc_episode()
action_word = {0:'up',1:'down',2:'left',3:'right'}

print(Gridmap.tell_policy())
print('==================')
for i in range(len(states)-1):
    if i == len(states):
        print('at s%s go %s landed at last state'%(states[i],action_word[actions[i]]))
        break
        
    print('at s%s go %s landed at s%s and got %s'%(states[i],action_word[actions[i]],states[i+1],rewards[i]))
    
print('got %s reward for ending after s%s'%(rewards[len(rewards)-1],states[len(states)-1]))

on state = 0 , do action = up
on state = 1 , do action = right
on state = 2 , do action = right
on state = 3 , do action = up
on state = 4 , do action = left
on state = 5 , do action = up
on state = 6 , do action = up
on state = 7 , do action = up
on state = 8 , do action = up
on state = 9 , do action = left
on state = 10 , do action = up
None
at s6 go up landed at s2 and got -1
at s2 go right landed at s6 and got -1
at s6 go up landed at s8 and got -1
at s8 go up landed at s6 and got -1
at s6 go up landed at s2 and got -1
got 10 reward for ending after s2


In [15]:
Gridmap.policy

array([[1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.],
       [0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [16]:
for state in range(Gridmap.action_value.shape[1]):
    print('value of s%s: up: %s, down: %s, left: %s, right: %s'%(state,Gridmap.action_value[0,state],Gridmap.action_value[1,state],Gridmap.action_value[2,state],Gridmap.action_value[3,state]))

value of s0: up: 0.0, down: 0.0, left: 0.0, right: 0.0
value of s1: up: -0.1709568587381877, down: -0.16867573416580217, left: -0.6635417520329808, right: -0.06900226030585666
value of s2: up: -0.09250346739810446, down: -0.12412168030957911, left: -0.12077595496855949, right: -0.008867438825192898
value of s3: up: 0.0, down: 0.0, left: 0.0, right: 0.0
value of s4: up: -0.1342282867806504, down: -0.18103350090972775, left: -0.04182097062153927, right: -0.1290167929882382
value of s5: up: -0.08234129769299624, down: -0.16579733022479354, left: -0.16565707216007428, right: -0.6573631472419137
value of s6: up: -0.13034785988961023, down: -0.33826545880287817, left: -0.3256543967116718, right: -0.8120276744779945
value of s7: up: -0.19986550249376767, down: -0.4421853320505261, left: -0.4311619550784326, right: -0.8671645464545591
value of s8: up: -0.18481117626509067, down: -0.3803007844405331, left: -0.9349681592769523, right: -0.47666848216450114
value of s9: up: -0.6999915881075139, do